In [1]:
cd /home/tvangraft/tudelft/thesis/metaengineering

/home/tvangraft/tudelft/thesis/metaengineering


In [5]:
from collections import defaultdict
from typing import DefaultDict, List, Hashable, Dict, Any

from src.utils.utils import get_generator, get_project_root
from src.utils.test_result_store import TestResultStore

from src.pipeline.config import DataLoaderConfig, TaskLoaderConfig
from src.pipeline.taskloader import TaskLoader, TaskFrame
from src.pipeline.dataloader import DataLoader

from src.orchestrator.trainer import Trainer

from src.settings.tier import Tier
from src.settings.strategy import Strategy
from src.settings.metabolites import ENZYMES, METABOLITES, PRECURSOR_METABOLITES, PRECURSOR_METABOLITES_NO_TRANSFORM

from src.gnn.data_augmentation import DataAugmentation
from src.gnn.embeddings import generate_embedding
from src.gnn.graph_builder import get_samples_hetero_graph, get_graph_fc

import pandas as pd
import numpy as np

import cobra
from cobra.util import create_stoichiometric_matrix
from cobra.core import Reaction

import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

from more_itertools import flatten

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_absolute_error

from scipy.stats import pearsonr

In [3]:
path = "/home/tvangraft/tudelft/thesis/metaengineering/data"
model = cobra.io.read_sbml_model(f'{path}/iMM904.xml')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [6]:
edge_list_df_unfiltered = pd.read_csv('./data/training/edge_list_unfiltered.csv')
graph_fc_unfiltered = get_graph_fc(edge_list_df_unfiltered, PRECURSOR_METABOLITES_NO_TRANSFORM)

edge_list_df_strict = pd.read_csv('./data/training/edge_list_strict.csv')
graph_fc_strict = get_graph_fc(edge_list_df_strict, PRECURSOR_METABOLITES)

edge_list_df_all = pd.read_csv('./data/training/edge_list_all.csv')
graph_fc_all = get_graph_fc(edge_list_df_all, METABOLITES)

KeyError: '2pg'

In [15]:
graph_fc_df = get_graph_fc_df(edge_list_df_unfiltered, METABOLITES)
graph_fc_df.std()

YBL015W    1.005305
YBR026C    1.005305
YBR221C    1.005305
YCR005C    1.005305
YDL040C    1.005305
             ...   
e4p        0.764922
f6p        1.005305
oaa        0.437595
pyr        1.005305
r5p        1.005305
Length: 77, dtype: float64

In [16]:
split_kwargs = dict(
    stratify='metabolite_id',
    shuffle=True
)
X_train, X_test, y_train, y_test = trainer.do_train_test_split(tf, Strategy.ALL, **split_kwargs)


In [17]:
h = np.array(X_test['KO_ORF'].unique())
k = np.array(X_test[X_test['metabolite_id'] == 'pyr']['KO_ORF'].unique())

np.argwhere(np.isin(h, k)).flatten().shape

(29,)

# Modeling

In [18]:
import os
from functools import partial

from tqdm import tqdm

import torch
from torch.nn import BatchNorm1d, ModuleList
import torch.nn.functional as F

from torch_geometric.data import Data, HeteroData
from torch_geometric.utils import from_networkx, to_networkx
from torch_geometric.loader import DataLoader as GeoDataLoader
from torch_geometric.nn import GAT, GCNConv, to_hetero, SAGEConv, GATConv, HeteroLinear, Linear, Node2Vec
import torch_geometric.transforms as T
from torch_geometric.nn.conv import HeteroConv

import mlflow.pytorch

from config import HYPERPARAMETERS, BEST_PARAMETERS

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.air import session, RunConfig
from ray.tune.integration.mlflow import mlflow_mixin
from ray.tune.integration.mlflow import MLflowLoggerCallback

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
mlflow.set_tracking_uri("http://localhost:5000")
device = torch.device("cpu")
torch.manual_seed(42)

/home/tvangraft/tudelft/thesis/.env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data preparation

## Model prep

In [ ]:
def count_parameters(model):
    # for p in model.parameters():
    #     print(p)

    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def log_metrics(all_preds, all_ground_truth, all_knockout_ids, epoch, type: str, debug=False):
    mae = mean_absolute_error(all_ground_truth, all_preds)
    r2 = pearsonr(all_preds, all_ground_truth)[0]
    # all_knockout_ids = [data_augmentation_get_knockout_label(knockout_id) for knockout_id in all_knockout_ids]
    
    # k = np.array(X_test[X_test['metabolite_id'] == 'pyr']['KO_ORF'].unique())
    # mask_idx = np.argwhere(np.isin(all_knockout_ids, k)).flatten()
    # masked_mae = mean_absolute_error(all_ground_truth[mask_idx], all_preds[mask_idx])
    # masked_r2 = pearsonr(all_preds[mask_idx], all_ground_truth[mask_idx])[0]
    
    if debug:
        print(f"{mae=}")
        print(f"{r2=}")
    mlflow.log_metric(key="Mean absolute error", value=float(mae), step=epoch)
    mlflow.log_metric(key="R2 score", value=float(r2), step=epoch)
    # mlflow.log_metric(key="Masked Mean absolute error", value=float(masked_mae), step=epoch)
    # mlflow.log_metric(key="Masked R2 score", value=float(masked_r2), step=epoch)

In [ ]:
def train_one_epoch(epoch, model, train_loader, optimizer, loss_fn):
    # Enumerate over the data
    running_loss = 0.0
    step = 0
    for _, batch in enumerate(train_loader):
        # Use GPU
        batch.to(device)
        # Reset gradients
        optimizer.zero_grad() 
        # Passing the node features and the connection info
        pred = model.forward(
            batch.x_dict, 
            batch.edge_index_dict,
            # batch.edge_attr.float(),
            # batch.batch
        ) 
        # Calculating the loss and gradients
        train_mask = batch['metabolites'].train_mask.bool()
        loss = loss_fn(
            torch.squeeze(pred['metabolites']), 
            batch['metabolites'].y.float()
        )

        loss.backward()  
        optimizer.step()  
        # Update tracking
        running_loss += loss.item()
        step += 1
    
    with tune.checkpoint_dir(epoch) as checkpoint_dir:
        path = os.path.join(checkpoint_dir, "checkpoint")
        torch.save((model.state_dict(), optimizer.state_dict()), path)

    tune.report(loss=(running_loss/step))

    return running_loss/step
    
@mlflow_mixin
def test(epoch, model, test_loader, loss_fn, debug=False):
    all_preds_raw = []
    all_labels = []
    all_knockout_ids = []
    running_loss = 0.0
    step = 0
    for batch in test_loader:
        batch.to(device)
        # print(batch)  
        pred = model(
            batch.x_dict, 
            batch.edge_index_dict,
        )
        test_mask = batch['metabolites'].test_mask.bool()

        if debug:
            print(f"{test_mask.sum()=}")
            print(
                f"{pred['metabolites'].shape=} \n"
                f"{pred['metabolites'][test_mask].mean()=} \n"
                f"{pred['metabolites'][test_mask].max()=} \n"
                f"{pred['metabolites'][test_mask].min()=} \n"
                f"{pred['metabolites'][test_mask].shape=} \n"
            ) 
        
        loss = loss_fn(
            torch.squeeze(pred['metabolites'][test_mask]), 
            torch.squeeze(batch['metabolites'].y.float()[test_mask])
        )

         # Update tracking
        running_loss += loss.item()
        step += 1
        all_preds_raw.append(pred['metabolites'][test_mask].cpu().detach().numpy())
        all_labels.append(batch['metabolites'].y[test_mask].cpu().detach().numpy())
        # all_knockout_ids.append(batch['enzymes'].knockout_label_id.cpu().detach().numpy())
    
    all_preds_raw = np.concatenate(all_preds_raw).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    # all_knockout_ids = np.concatenate(all_knockout_ids).ravel()
    log_metrics(all_preds_raw, all_labels, all_knockout_ids, epoch, "test")
    return running_loss/step

## Model construction

In [ ]:
class HeteroGCNModel(torch.nn.Module):
    def __init__(self, model_config) -> None:
        super(HeteroGCNModel, self).__init__()
        embedding_size = model_config["model_embedding_size"]
        n_heads = model_config["model_attention_heads"]
        self.n_layers = model_config["model_layers"]
        
        self.conv_layers = ModuleList([])
        self.transf_layers = ModuleList([])
        self.pooling_layers = ModuleList([])
        self.bn_layers = ModuleList([])
        
        self.conv1 = GATConv(
            (-1, -1), out_channels=embedding_size, heads=n_heads, add_self_loops=False, bias=False
        )
        self.transf1 = Linear(
            in_channels=embedding_size*n_heads, 
            out_channels=embedding_size, 
        )

        for i in range(self.n_layers):
            self.conv_layers.append(
                GATConv(
                    (-1, -1), 
                    out_channels=embedding_size, 
                    heads=n_heads, 
                    add_self_loops=False
                )
            )
            self.transf_layers.append(
                Linear(
                    embedding_size*n_heads, 
                    embedding_size
                )
            )

        self.conv2 = GATConv(
            (-1, -1), out_channels=1, add_self_loops=False, bias=False
        )

        self.linear1 = Linear(n_heads * embedding_size, embedding_size)
        self.linear2 = Linear(embedding_size, 1)
    
    def forward(self, x, edge_index):
        # random weights for metabolite nodes should cancel out their contribution
        # metabolite_fc = torch.rand(x.shape[0], device=device)
        
        x = self.conv1(x, edge_index)
        # print(x)
        # x = torch.relu(self.transf1(x))

        for i in range(self.n_layers):
            x = self.conv_layers[i](x, edge_index)
            x = torch.relu(self.transf_layers[i](x))

        x = self.conv2(x, edge_index)
        
        # x = torch.relu(self.linear1(x))
        # x = F.dropout(x, p=0.8, training=self.training)
        # x = self.linear2(x)
        return x 

## Model running

In [ ]:
model_config = {
    "model_embedding_size": 64,
    "model_attention_heads": 3,
    "model_layers": 5,
    "batch_size": 4,
    "learning_rate": 0.01,
    "sgd_momentum": 0.8,
    "scheduler_gamma": 1,
}

@mlflow_mixin
def run_one_training(model_config, metabolite_id, checkpoint_dir):
    print(f"creating model {model_config=}")
    samples = get_samples_hetero_graph(metabolite_id)
    params = model_config
    run_id = model_config['mlflow']['tags']['mlflow.parentRunId']
    with mlflow.start_run(nested=True) as run:
        mlflow.set_tag("mlflow.parentRunId", run_id)
        # Logging params
        for key in params.keys():
            mlflow.log_param(key, params[key])

        # Preparing training
        train_loader = GeoDataLoader(samples, batch_size=params['batch_size'])
        test_loader = GeoDataLoader(samples, batch_size=1)
        
        # Loading the model
        print("Loading model...")
        model = HeteroGCNModel(model_config=params)
        model = to_hetero(model, samples[0].metadata(), aggr='mean')
        model = model.to(device)

        if checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(checkpoint_dir, "checkpoint"))
            model.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)

        # < 1 increases precision, > 1 recall
        # loss_fn = torch.nn.MSELoss(reduction='none')
        loss_fn = torch.nn.MSELoss()
        # we need to keep the lr quite low since otherwise the weights explode
        optimizer = torch.optim.SGD(
            model.parameters(), 
            lr=params['learning_rate'],
            momentum=params['sgd_momentum'],
            # weight_decay=5e-4
        )
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=params['scheduler_gamma'])
        
        # Start training
        best_loss = 1000
        early_stopping_counter = 0
        max_epochs = 1
        for epoch in tqdm(range(max_epochs)): 
            if early_stopping_counter <= 10: # = x * 5 
                # Training
                model.train()
                loss = train_one_epoch(epoch, model, train_loader, optimizer, loss_fn)
                # print(f"Epoch {epoch} | Train Loss {loss}")
                mlflow.log_metric(key="Train loss", value=float(loss), step=epoch)

                # Testing
                model.eval()
                if epoch % 5 == 0 or epoch == max_epochs - 1:
                    loss = test(epoch, model, test_loader, loss_fn)
                    # print(f"Epoch {epoch} | Test Loss {loss}")
                    mlflow.log_metric(key="Test loss", value=float(loss), step=epoch)
                    
                    # Update best loss
                    if float(loss) < best_loss:
                        best_loss = loss
                        # Save the currently best model 
                        # mlflow.pytorch.log_model(model, "model", signature=SIGNATURE)
                        early_stopping_counter = 0
                    else:
                        early_stopping_counter += 1

                scheduler.step()
                mlflow.log_metric(key="Learning rate", value=float(scheduler.get_last_lr()[0]), step=epoch)
            else:
                print("Early stopping due to no improvement.")
                session.report({
                    "loss": best_loss
                })
                return {"loss": best_loss}
    print(f"Finishing training with best test loss: {best_loss}")

    with torch.no_grad():
        sample = samples[0].to(device)
        model.forward(sample.x_dict, sample.edge_index_dict)
        print(f"Number of parameters: {count_parameters(model)}")

    session.report({
        "loss": best_loss
    })
    return {"loss": best_loss}

In [40]:
@mlflow_mixin
def run_one_training(model_config, metabolite_id, checkpoint_dir):
    print(f"creating model {model_config=}")

    samples = get_samples_hetero_graph(metabolite_id)
    params = model_config
    run_id = model_config['mlflow']['tags']['mlflow.parentRunId']
    mlflow.set_tag("mlflow.parentRunId", run_id)
    # Logging params
    for key in params.keys():
        mlflow.log_param(key, params[key])

    # Preparing training
    train_loader = GeoDataLoader(samples, batch_size=params['batch_size'])
    test_loader = GeoDataLoader(samples, batch_size=1)
    
    # Loading the model
    print("Loading model...")
    model = HeteroGCNModel(model_config=params)
    model = to_hetero(model, samples[0].metadata(), aggr='mean')
    model = model.to(device)

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    # < 1 increases precision, > 1 recall
    # loss_fn = torch.nn.MSELoss(reduction='none')
    loss_fn = torch.nn.MSELoss()
    # we need to keep the lr quite low since otherwise the weights explode
    optimizer = torch.optim.SGD(
        model.parameters(), 
        lr=params['learning_rate'],
        momentum=params['sgd_momentum'],
        # weight_decay=5e-4
    )
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=params['scheduler_gamma'])
    
    # Start training
    best_loss = 1000
    early_stopping_counter = 0
    max_epochs = 300
    for epoch in tqdm(range(max_epochs)): 
        if early_stopping_counter <= 25: # = x * 5 
            # Training
            model.train()
            loss = train_one_epoch(epoch, model, train_loader, optimizer, loss_fn)
            # print(f"Epoch {epoch} | Train Loss {loss}")
            mlflow.log_metric(key="Train loss", value=float(loss), step=epoch)
            
            # Testing
            model.eval()
            if epoch % 5 == 0 or epoch == max_epochs - 1:
                loss = test(epoch, model, test_loader, loss_fn)
                # print(f"Epoch {epoch} | Test Loss {loss}")
                mlflow.log_metric(key="Test loss", value=float(loss), step=epoch)
                
                # Update best loss
                if float(loss) < best_loss:
                    best_loss = loss
                    # Save the currently best model 
                    # mlflow.pytorch.log_model(model, "model", signature=SIGNATURE)
                    early_stopping_counter = 0
                else:
                    early_stopping_counter += 1

            scheduler.step()
            mlflow.log_metric(key="Learning rate", value=float(scheduler.get_last_lr()[0]), step=epoch)
            
        else:
            print("Early stopping due to no improvement.")
            session.report({
                "loss": best_loss
            })
            return {"loss": best_loss}
    print(f"Finishing training with best test loss: {best_loss}")

    with torch.no_grad():
        sample = samples[0].to(device)
        model.forward(sample.x_dict, sample.edge_index_dict)
        print(f"Number of parameters: {count_parameters(model)}")

    session.report({
        "loss": best_loss
    })

    mlflow.end_run()

    return {"loss": best_loss}

In [36]:
HYPERPARAMETERS = {
    "batch_size": tune.choice([2, 4, 8]),
    "learning_rate": tune.choice([0.1, 0.05, 0.01, 0.001]),
    "sgd_momentum": tune.choice([0.9, 0.8, 0.5]),
    "scheduler_gamma": tune.choice([0.995, 0.9, 0.8, 0.5, 1]),
    "model_embedding_size": tune.choice([8, 16, 32, 64, 128]),
    "model_attention_heads": tune.choice([1, 2, 3, 4]),
    "model_layers": tune.choice([1, 3, 5, 7]),
}

In [43]:
def tune_metabolite_hyper_parameters(metabolite_id, experiment_name, num_samples=10):
    # mlflow.set_tracking_uri(mlflow.get_tracking_uri())
    # mlflow.set_experiment(experiment_name=experiment_name)

    parent_id = mlflow.active_run().info.run_id if mlflow.active_run() else None

    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=300,
        grace_period=10,
        reduction_factor=2
    )
    reporter = CLIReporter(
        metric_columns=["loss", "training_iteration"]
    )

    trainable = tune.with_parameters(
        run_one_training, 
        metabolite_id=metabolite_id,
        checkpoint_dir=None,
    )

    result = tune.run(
        trainable,
        # resources_per_trial={"cpu": 1, "gpu": 1},
        config={
            **HYPERPARAMETERS,
            "mlflow": {
                "experiment_name": experiment_name,
                "tracking_uri": mlflow.get_tracking_uri(),
                "save_artifacts": True,
                "tags": {
                    "mlflow.parentRunId": parent_id
                }
            }
        },
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter,
    )
    return result

# tune_metabolite_hyper_parameters("pyr")

In [45]:
run_name = "model_gat_node_embeddings"
experiment_name = 'metabolite_gnn_sweep_full'
mlflow.set_experiment(experiment_name)
with mlflow.start_run(run_name=run_name) as run:
    for metabolite_id in PRECURSOR_METABOLITES:
        with mlflow.start_run(run_name=f"model_{metabolite_id}", nested=True):
            result = tune_metabolite_hyper_parameters(metabolite_id, experiment_name, num_samples=16)
            print(result)

== Status ==
Current time: 2022-12-11 18:34:34 (running for 00:00:01.17)
Memory usage on this node: 5.6/23.5 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 160.000: None | Iter 80.000: None | Iter 40.000: None | Iter 20.000: None | Iter 10.000: None
Resources requested: 1.0/16 CPUs, 0/0 GPUs, 0.0/11.6 GiB heap, 0.0/5.8 GiB objects
Result logdir: /home/tvangraft/ray_results/run_one_training_2022-12-11_18-34-33
Number of trials: 16/16 (15 PENDING, 1 RUNNING)
+------------------------------+----------+--------------------+--------------+-----------------+------------------------+------------------------+----------------+-------------------+----------------+
| Trial name                   | status   | loc                |   batch_size |   learning_rate |   model_attention_head |   model_embedding_size |   model_layers |   scheduler_gamma |   sgd_momentum |
|                              |          |                    |              |                 |                      s |     

  0%|          | 0/300 [00:00<?, ?it/s]0m 


Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
run_one_training_136e7_00000,2022-12-11_18-35-25,True,,33c6689ea163442ca46f502cea599b39,VGI-DEKSPTOP-TIJMEn,40,0.691313,172.29.86.26,26578,True,47.687,0.644296,47.687,1670780125,0,,40,136e7_00000,0.124693
run_one_training_136e7_00001,2022-12-11_18-35-05,True,,8c379741491d4b0da3f0197a8787f06b,VGI-DEKSPTOP-TIJMEn,20,0.703326,172.29.86.26,26689,True,22.3516,0.58685,22.3516,1670780105,0,,20,136e7_00001,0.252936
run_one_training_136e7_00002,2022-12-11_18-34-52,True,,9a332a3f9fab4c649ca68d9e04a0bc96,VGI-DEKSPTOP-TIJMEn,10,0.704342,172.29.86.26,26691,True,9.32165,0.667562,9.32165,1670780092,0,,10,136e7_00002,0.2263
run_one_training_136e7_00003,2022-12-11_18-34-52,True,,f9aa2418ace5448cbb31744b81fd76cd,VGI-DEKSPTOP-TIJMEn,10,0.70405,172.29.86.26,26693,True,9.23457,0.63072,9.23457,1670780092,0,,10,136e7_00003,0.210002
run_one_training_136e7_00004,2022-12-11_18-35-19,True,,7a3fd7948a0b46fbbdac1a253d8d2e05,VGI-DEKSPTOP-TIJMEn,40,0.693243,172.29.86.26,26695,True,36.3528,0.560302,36.3528,1670780119,0,,40,136e7_00004,0.20078
run_one_training_136e7_00005,2022-12-11_18-35-14,True,,a656105661434dffbb10d7de2e7fe45e,VGI-DEKSPTOP-TIJMEn,10,0.703734,172.29.86.26,26697,True,31.3902,2.07448,31.3902,1670780114,0,,10,136e7_00005,0.225905
run_one_training_136e7_00006,2022-12-11_18-34-53,True,,74c3bc93e7bd4e839ea19b40efb29a75,VGI-DEKSPTOP-TIJMEn,10,0.704402,172.29.86.26,26699,True,10.4462,0.529096,10.4462,1670780093,0,,10,136e7_00006,0.292018
run_one_training_136e7_00007,2022-12-11_18-35-03,True,,a2ac2ecb4070484085363b80edd90e1a,VGI-DEKSPTOP-TIJMEn,10,0.705958,172.29.86.26,26701,True,20.2915,1.31262,20.2915,1670780103,0,,10,136e7_00007,0.22197
run_one_training_136e7_00008,2022-12-11_18-35-04,True,,d1ebbb18434a4d4fa3aec636ab13cb94,VGI-DEKSPTOP-TIJMEn,20,0.703275,172.29.86.26,26703,True,21.246,0.55011,21.246,1670780104,0,,20,136e7_00008,0.231165
run_one_training_136e7_00009,2022-12-11_18-34-59,True,,929ebbe38b7749a28031ea0ca62fe7a9,VGI-DEKSPTOP-TIJMEn,10,0.703586,172.29.86.26,26706,True,16.4653,1.18474,16.4653,1670780099,0,,10,136e7_00009,0.270522


(run_one_training pid=26578) /home/tvangraft/tudelft/thesis/.env/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
(run_one_training pid=26578)   warnings.warn(stats.ConstantInputWarning(msg))
  0%|          | 1/300 [00:03<15:34,  3.12s/it]


(run_one_training pid=26689) creating model model_config={'batch_size': 4, 'learning_rate': 0.01, 'sgd_momentum': 0.5, 'scheduler_gamma': 0.5, 'model_embedding_size': 64, 'model_attention_heads': 3, 'model_layers': 3, 'mlflow': {'experiment_name': 'metabolite_gnn_sweep_full', 'tracking_uri': 'http://localhost:5000', 'save_artifacts': True, 'tags': {'mlflow.parentRunId': '90a5008d4e44448ea3188234b4f04f36'}}}
(run_one_training pid=26708) creating model model_config={'batch_size': 4, 'learning_rate': 0.01, 'sgd_momentum': 0.5, 'scheduler_gamma': 1, 'model_embedding_size': 64, 'model_attention_heads': 2, 'model_layers': 5, 'mlflow': {'experiment_name': 'metabolite_gnn_sweep_full', 'tracking_uri': 'http://localhost:5000', 'save_artifacts': True, 'tags': {'mlflow.parentRunId': '90a5008d4e44448ea3188234b4f04f36'}}}
(run_one_training pid=26716) creating model model_config={'batch_size': 8, 'learning_rate': 0.01, 'sgd_momentum': 0.5, 'scheduler_gamma': 0.8, 'model_embedding_size': 8, 'model_att

...

== Status ==
Current time: 2022-12-11 18:52:43 (running for 00:03:32.39)
Memory usage on this node: 4.8/23.5 GiB 
Using AsyncHyperBand: num_stopped=10
Bracket: Iter 160.000: -0.6931263001558061 | Iter 80.000: -0.6913105995239069 | Iter 40.000: -0.6913367883923153 | Iter 20.000: -0.6931748799979687 | Iter 10.000: -0.7021237503116329
Resources requested: 1.0/16 CPUs, 0/0 GPUs, 0.0/11.6 GiB heap, 0.0/5.8 GiB objects
Result logdir: /home/tvangraft/ray_results/run_one_training_2022-12-11_18-49-11
Number of trials: 16/16 (1 RUNNING, 15 TERMINATED)
+------------------------------+------------+--------------------+--------------+-----------------+------------------------+------------------------+----------------+-------------------+----------------+----------+----------------------+
| Trial name                   | status     | loc                |   batch_size |   learning_rate |   model_attention_head |   model_embedding_size |   model_layers |   scheduler_gamma |   sgd_momentum |     loss |

 45%|████▌     | 135/300 [03:27<02:50,  1.03s/it]


(run_one_training pid=11337) Early stopping due to no improvement.


 45%|████▌     | 136/300 [03:28<04:11,  1.54s/it]
2022-12-11 18:52:48,519	INFO tune.py:777 -- Total run time: 217.18 seconds (217.04 seconds for the tuning loop).


== Status ==
Current time: 2022-12-11 18:52:48 (running for 00:03:37.05)
Memory usage on this node: 4.6/23.5 GiB 
Using AsyncHyperBand: num_stopped=10
Bracket: Iter 160.000: -0.6931263001558061 | Iter 80.000: -0.6913105995239069 | Iter 40.000: -0.6913367883923153 | Iter 20.000: -0.6931748799979687 | Iter 10.000: -0.7021237503116329
Resources requested: 0/16 CPUs, 0/0 GPUs, 0.0/11.6 GiB heap, 0.0/5.8 GiB objects
Result logdir: /home/tvangraft/ray_results/run_one_training_2022-12-11_18-49-11
Number of trials: 16/16 (16 TERMINATED)
+------------------------------+------------+--------------------+--------------+-----------------+------------------------+------------------------+----------------+-------------------+----------------+----------+----------------------+
| Trial name                   | status     | loc                |   batch_size |   learning_rate |   model_attention_head |   model_embedding_size |   model_layers |   scheduler_gamma |   sgd_momentum |     loss |   training_i